In [1]:
import pandas as pd, numpy as np
from choicemodels.tools import distancematrix as dm

tract_centroids_file = '../data/bay_tract_centroids.csv'
distance_matrix_file = '../data/bay_tracts_distance_matrix.csv'

C:\Anaconda\envs\py2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## Load the data

In [2]:
# use double-precision floating points to ensure sufficient significant digits
dtypes = {'GEOID10':str, 'lat':np.float64, 'lng':np.float64}
df = pd.read_csv(tract_centroids_file, dtype=dtypes).sort_values(by='GEOID10')
len(df)

1588

In [3]:
# alternatively, create a randomized dataframe of length n to test performance relative to size
#n = 10000
#df = pd.DataFrame({'GEOID10':range(n), 'lng':np.random.random(n), 'lat':np.random.random(n)})

In [4]:
# index the dataframe by place identifier (i.e., census tract ID)
df = df.set_index('GEOID10')

## Calculate distance matrices

First, the euclidean distance matrix in units of degrees

In [5]:
%%time
df_eu_dm = dm.distance_matrix(df, method='euclidean')

Wall time: 34 ms


In [6]:
print(df_eu_dm.shape)
print(df_eu_dm.size)
df_eu_dm.iloc[:5, :5]

(1588, 1588)
2521744


,06001400100,06001400200,06001400300,06001400400,06001400500
06001400100,0.000000,0.026261,0.035165,0.032078,0.037980
06001400200,0.026261,0.000000,0.008998,0.007887,0.015172
06001400300,0.035165,0.008998,0.000000,0.008192,0.012998
06001400400,0.032078,0.007887,0.008192,0.000000,0.007288
06001400500,0.037980,0.015172,0.012998,0.007288,0.000000


Next, the great circle distance matrix in units of meters

In [7]:
%%time
df_gc_dm = dm.distance_matrix(df, method='greatcircle')

g:\geoff\dropbox\documents\school\phd\work\2017-summer\paul\code\choicemodels\choicemodels\tools\distancematrix.py:43: RuntimeWarning: invalid value encountered in arccos
  arc = np.arccos(cos)


Wall time: 1.47 s


In [8]:
print(df_gc_dm.shape)
print(df_gc_dm.size)
df_gc_dm.iloc[:5, :5]

(1588, 1588)
2521744


,06001400100,06001400200,06001400300,06001400400,06001400500
06001400100,0,2659,3595,3111,3579
06001400200,2659,0,944,692,1332
06001400300,3595,944,0,887,1262
06001400400,3111,692,887,0,640
06001400500,3579,1332,1262,640,0


In [9]:
# save distance matrix to disk
df_gc_dm.to_csv(distance_matrix_file, index=True, encoding='utf_8')